# Introduction

This notebook tests out some reasoning ideas with z3.

The main thing I want to know is whether we can derive properties of operations based on their definitions.
e.g. join implies contains.

In [57]:
import z3
from z3 import *

In [4]:
# Define the variables
# A table is a set of (i, j, v) tuples
Triple, mk_triple, (i_field, j_field, v_field) = z3.TupleSort(
    'Triple', [z3.IntSort(), z3.IntSort(), z3.IntSort()]
)

# Make a set sort whose elements are Triples
SetOfTriples = z3.SetSort(Triple)

In [5]:
S = z3.Const('S', SetOfTriples)

In [55]:
SetOfValues = z3.SetSort(z3.IntSort())
ProjectV = z3.Function('ProjectV', SetOfTriples, SetOfValues)
A = z3.Const('A', SetOfTriples)
V = z3.Const('V', SetOfValues)
i, j, v = z3.Ints('i j v')
t = mk_triple(i, j, v)
#i2, j2, = z3.Ints('i2 j2')
#t2 = mk_triple(i2, j2, v)

# v ∈ ProjectV(A)  iff  ∃ i,j . Triple(i,j,v) ∈ A
axiom_proj = z3.ForAll([A, i, j, v],
    z3.IsMember(v, ProjectV(A)) == z3.Exists([i, j], z3.IsMember(t, A)),
    patterns = [ z3.IsMember(t, A) ]
)
B = z3.Const('B', SetOfTriples)
# Define contains
Contains = z3.Function('Contains', SetOfTriples,   SetOfTriples,  z3.BoolSort())
# A ⊆ B iff ∀ v . v ∈ ProjectV(A) ⇒ ∃ i,j . Triple(i,j,v) ∈ B
axiom_contains = z3.ForAll([A, B],
    Contains(A, B) == z3.IsSubset(ProjectV(A), ProjectV(B))
)
contains_implies_subset = z3.ForAll([A, B],
    z3.Implies(Contains(A, B), z3.IsSubset(ProjectV(A), ProjectV(B)))
)

In [56]:
s = z3.Solver()
s.set(auto_config=False, mbqi=False)
s.add(axiom_proj)
s.add(axiom_contains)
s.add(contains_implies_subset)
s.check()
s.reason_unknown()

'smt tactic failed to show goal to be sat/unsat (incomplete (theory array))'

In [60]:
g = Goal()
g.add(axiom_proj, axiom_contains, contains_implies_subset)

#t = Then( Tactic('simplify'),
#          Tactic('qe'),
#          Tactic('smt') )
t = Then(
    Tactic('simplify'),        # rewrite and pull out simple eqs
    Tactic('solve-eqs'),       # solve small systems of equations
    Tactic('elim-term-ite'),   # remove “if-then-else” terms
    Tactic('array_blast'),     # Ackermannize/eliminate local array terms
    Tactic('qe'),              # quantifier elimination
    Tactic('ctx-solver-simplify')  # more simplification before final check
)
res = t(g)

print(res)

Z3Exception: unknown tactic 'array-blast'

In [ ]:


# Define the contains predicate
Contains = z3.Function('Contains',
                       SetOfTriples,   # domain
                       SetOfTriples,   # domain
                       z3.BoolSort())     # range

A, B = z3.Consts('A B', SetOfTriples)
i, j, v = z3.Ints('i j v')
t = mk_triple(i, j, v)
i2, j2, = z3.Ints('i2 j2')
t2 = mk_triple(i2, j2, v)
axiom = z3.ForAll([A, B, i, j, v],
                  Contains(A, B) ==
                  z3.Implies(z3.IsMember(t, A),
                             z3.Exists([i2, j2], z3.IsMember(t2, B))
                             ),
                  patterns = [
                      z3.Exists([i2, j2], z3.IsMember(t2, B)),
                      z3.Exists([i, j], z3.IsMember(t, A)),
                      z3.IsMember(t, A),
                      z3.IsMember(t2, B),
                  ],
                  )


In [24]:

s = z3.Solver()
s.set(auto_config=False, mbqi=False)
s.add(axiom)
s.check()

unknown

In [25]:
s.reason_unknown()

'smt tactic failed to show goal to be sat/unsat (incomplete (theory array))'

In [16]:
# Now show that Subset implies contains
theorem = z3.ForAll([A, B], z3.Implies(z3.IsSubset(A, B), Contains(A, B)))
s.push()
s.add(theorem)

# Check if the theorem is valid
s.check()

unknown

In [17]:
s.reason_unknown()

'(incomplete (theory array))'

# CVC5 with Z3 interface
Now try a solver with explicit relational theory.

In [61]:
import cvc5.pythonic as cvp

In [86]:

# 1) Create and configure the solver
s = cvp.Solver()
#s.setLogic("ALL")               # enable all logics, including sets & relations
s.setOption("produce-models", True)
s.setOption("finite-model-find", True)
s.setOption("sets-exp", True)

# 2) Declare sorts: integer, 3-tuple of integers, and sets of those tuples
Int = cvp.IntSort()  # the SMT integer sort
# TupleSort returns: the datatype, its constructor, and a tuple of field‐accessors
Triple, mk_triple, (proj_i, proj_j, proj_v) = cvp.TupleSort("Triple", [Int, Int, Int])  # :contentReference[oaicite:0]{index=0}
Rel3 = cvp.SetSort(Triple)                                                            # :contentReference[oaicite:1]{index=1}

# 3) Relation variables A, B : Set(Triple)
A = cvp.Const("A", Rel3)
B = cvp.Const("B", Rel3)

# 4) An uninterpreted “projection” function from Rel3 → Set(Int)
ProjectV = cvp.Function("ProjectV", Rel3, cvp.SetSort(Int))

# 5) Axiom: ProjectV(A) collects exactly the third components of the tuples in A
#    ∀A,i,j,v. v∈ProjectV(A) ↔ ∃i2,j2. (i2,j2,v)∈A
i, j, v, i2, j2 = cvp.Ints("i j v i2 j2")                                           # :contentReference[oaicite:2]{index=2}
t2 = mk_triple(i2, j2, v)
s.add(cvp.ForAll([A, i, j, v],
             cvp.IsMember(v, ProjectV(A))  # v ∈ ProjectV(A)
             ==
             cvp.Exists([i2, j2], cvp.IsMember(t2, A))  # ∃ i2,j2. (i2,j2,v) ∈ A
            ))                                                                         # :contentReference[oaicite:3]{index=3}

# 6) Now assert the “contains-implies-subset” check on the projected values:
#    ProjectV(A) ⊆ ProjectV(B)
#s.add(cvp.IsSubset(ProjectV(A), ProjectV(B)))                                        # :contentReference[oaicite:4]{index=4}
diff = cvp.SetDifference(ProjectV(A), ProjectV(B))
empty_set = cvp.EmptySet(Int)
s.add(cvp.IsSubset(diff, empty_set))  # ProjectV(A) ⊆ ProjectV(B


# 7) Check satisfiability and, if sat, print a witness model
res = s.check()
print("Result:", res)

Result: unknown


In [87]:
reason = s.reason_unknown()
reason, type(reason)

(<UnknownExplanation.INCOMPLETE: 1>, <enum 'UnknownExplanation'>)

In [104]:
from cvc5.pythonic import Solver, Kind, TupleSort, SetSort, Const, IntSort
import cvc5

tm = cvc5.TermManager()
s = cvp.Solver()
#s.setLogic("ALL")   # Solver' object has no attribute 'setLogic' (cvc5-1.2.1)
s.setOption("produce-models", True)
s.setOption("finite-model-find", True)
s.setOption("sets-exp", True)

Triple, mk_triple, (proj_i, proj_j, proj_v) = TupleSort("Triple", [IntSort(), IntSort(), IntSort()])
Rel3   = SetSort(Triple)

A = Const("A", Rel3)

#projOp    = tm.mkOp(Kind.RELATION_PROJECT, 3)
#ProjectV  = lambda R: tm.mkTerm(projOp, R)
ProjectV = lambda R: R.
cvp.Set

s.add(ProjectV(A).subsetOf(ProjectV(A)))
print("QF check:", s.check())


AttributeError: 'SetRef' object has no attribute 'project'

In [120]:
import cvc5
from cvc5 import Kind

# 1) Create and configure a base‐API solver
solver = cvc5.Solver()
solver.setLogic("QF_ALL")               # quantifier‐free fragment
solver.setOption("produce-models", "true")
solver.setOption("finite-model-find", "true")
solver.setOption("sets-exp", "true")    # enable extended set/rel support

# 2) Build the Int sort, the 3-tuple sort, and the set-of-tuples sort
Int    = solver.getIntegerSort()
#Int = cvp.IntSort()  # the SMT integer sort
Triple = tm.mkTupleSort(Int, Int, Int)   # Tuple(Int,Int,Int)
#Triple, mk_triple, (proj_i, proj_j, proj_v) = solver.mkTupleSort([Int, Int, Int])  # Tuple(Int,Int,Int)
#Triple, mk_triple, (proj_i, proj_j, proj_v) = cvp.TupleSort("Triple", [Int, Int, Int])  # :contentReference[oaicite:0]{index=0}
Rel3   = solver.mkSetSort(Triple)               # Set(Tuple(Int,Int,Int)) :contentReference[oaicite:0]{index=0}

# 3) Declare one relation variable A : Rel3
A = solver.mkConst(Rel3, "A")

# 4) Build the projection operator rel.project[3]
#    mkOp takes a Kind plus a list of uint32_t parameters
projOp = solver.mkOp(Kind.RELATION_PROJECT, 2)
#    Then mkTerm applies that operator to your set
ProjectV = lambda R: solver.mkTerm(projOp, R)

# 5) Now you can write a _quantifier‐free_ subset check:
#      rel.project[3](A) ⊆ rel.project[3](A)
subset_check = solver.mkTerm(
    Kind.SET_SUBSET,
     ProjectV(A),
      ProjectV(A)
)

# Define the contains predicate
Contains = solver.mkOp(Kind.R, 2)
# A ⊆ B iff ∀ v . v ∈ ProjectV(A) ⇒ ∃ i,j . Triple(i,j,v) ∈ B
axiom_contains = solver.mkTerm(
    Kind.FORALL,
    [A, B],
    solver.mkTerm(Kind.EQUAL, Contains(A, B),
                  solver.mkTerm(Kind.SET_SUBSET, ProjectV(A), ProjectV(B))
                  )
)

solver.assertFormula(axiom_contains)

# 6) This is a purely QF_SETS+REL fragment—no quantifiers—so CVC5 will decide it:
print("QF check:", solver.checkSat())    # → sat


AttributeError: RELATION_CONTAINS

In [137]:
import cvc5
from cvc5 import Kind

# 1) Solver setup in the quantifier‐free fragment
solver = cvc5.Solver()
#solver.setLogic("QF_ALL")
solver.setLogic("HO_ALL")                # ← enable higher-order support
solver.setOption("produce-models", "true")
solver.setOption("finite-model-find", "true")
solver.setOption("sets-exp", "true")

# 2) Sorts: Int, 3-tuple(Int,Int,Int), and Set(3-tuple)
Int    = solver.getIntegerSort()
Triple = tm.mkTupleSort(Int, Int, Int)
Rel3   = solver.mkSetSort(Triple)

# 3) One relation variable A : Rel3 (and B if you like)
A = solver.mkConst(Rel3, "A")
B = solver.mkConst(Rel3, "B")

# 4) Build rel.project[3] operator and helper
projOp   = solver.mkOp(Kind.RELATION_PROJECT, 2)
ProjectV = lambda R: solver.mkTerm(projOp, R)

# 5) Declare Contains as an uninterpreted function symbol
ContainsSort = solver.mkFunctionSort([Rel3, Rel3],
                                     solver.getBooleanSort())
ContainsSym  = solver.mkConst(ContainsSort, "Contains")

# 6) Build the two terms you want to equate:
#    a) Contains(A,B)
contains_AB = solver.mkTerm(Kind.APPLY_UF,
                            ContainsSym, A, B)

#    b) the projection-subset test: ProjectV(A) ⊆ ProjectV(B)
subset_AB = solver.mkTerm(Kind.SET_SUBSET,
                          ProjectV(A),
                           ProjectV(B))

# 7) Constrain them to be equal (still QF!)
solver.assertFormula(
    solver.mkTerm(Kind.EQUAL,
                  contains_AB, subset_AB)
)

# 8) Check & inspect the model
res = s.check()
print("Result:", res)             # prints “sat” / “unsat” / “unknown”

Result: sat


In [136]:
type(res.r)

cvc5.cvc5_python_base.Result

In [138]:
class Test: pass

In [141]:
from z3 import *

# 1) Domain sort
T = DeclareSort('T')

# 2) Binary-tuple sort for 2-ary relations
Tuple2 = Datatype('Tuple2')
Tuple2.declare('mk2', ('first', T), ('second', T))
Tuple2 = Tuple2.create()

# Relation sort for binary relations
Rel2 = SetSort(Tuple2)

# 3) 4-tuple sort for Cartesian products
Tuple4 = Datatype('Tuple4')
Tuple4.declare('mk4',
    ('f1', T), ('f2', T),   # from the first relation
    ('s1', T), ('s2', T))   # from the second relation
Tuple4 = Tuple4.create()

# Relation sort for 4-ary relations (i.e. R×S)
Rel4 = SetSort(Tuple4)

# 4) Declare the RA operators (all as uninterpreted functions)
Union  = Function('Union', Rel2,  Rel2,  Rel2)
Diff   = Function('Diff',  Rel2,  Rel2,  Rel2)
Prod   = Function('Prod',  Rel2,  Rel2,  Rel4)
Sigma1 = Function('Sigma1', T,     Rel2,  Rel2)      # σ_{first=c}(R)
Pi1    = Function('Pi1',    Rel2,  SetSort(T))       # π_{first}(R)

# 5) Axioms

# helper constants
x, y    = Consts('x y', Tuple2)
t4      = Const('t4', Tuple4)
c       = Const('c', T)
e       = Const('e', T)
R, S    = Consts('R S', Rel2)

axioms = []

# -- Union: x∈Union(R,S)  ⇔  x∈R ∨ x∈S
axioms.append(
    ForAll([x, R, S],
      IsMember(x, Union(R,S)) ==
      Or(IsMember(x, R), IsMember(x, S)))
)

# -- Difference: x∈Diff(R,S)  ⇔  x∈R ∧ x∉S
axioms.append(
    ForAll([x, R, S],
      IsMember(x, Diff(R,S)) ==
      And(IsMember(x, R), Not(IsMember(x, S))))
)

# -- Cartesian product:
#    t4 ∈ Prod(R,S)
#      ⇔ ∃u∈R, v∈S. t4 = mk4(u.first,u.second, v.first,v.second)
u, v = Consts('u v', Tuple2)
axioms.append(
    ForAll([t4, R, S],
      IsMember(t4, Prod(R,S)) ==
      Exists([u, v],
        And(
          IsMember(u, R),
          IsMember(v, S),
          t4 == Tuple4.mk4(
                  Tuple2.first(u), Tuple2.second(u),
                  Tuple2.first(v), Tuple2.second(v))
        )))
)

# -- Selection on first field: x∈Sigma1(c,R)  ⇔  x∈R ∧ x.first = c
axioms.append(
    ForAll([x, c, R],
      IsMember(x, Sigma1(c, R)) ==
      And(IsMember(x, R), Tuple2.first(x) == c))
)

# -- Projection on first field: e∈Pi1(R)  ⇔  ∃x∈R. x.first = e
axioms.append(
    ForAll([e, R],
      IsMember(e, Pi1(R)) ==
      Exists([x],
        And(IsMember(x, R), Tuple2.first(x) == e)))
)

# 6) Example: prove R ⊆ Union(R,S)
s = Solver()
s.add(axioms)

# formula: ∀x,R,S. x∈R ⇒ x∈Union(R,S)
phi = ForAll([x, R, S],
           Implies(IsMember(x, R),
                   IsMember(x, Union(R,S))))
# check ¬phi is unsat
s.push()
s.add(Not(phi))
print("Containment R ⊆ R∪S valid? =>", s.check())  # should print: unsat


Containment R ⊆ R∪S valid? => unsat


In [146]:
from z3 import *

# 1) Define a tiny AST for relational programs
RA = Datatype('RA')
RA.declare('Table',    ('name',   StringSort()))
#RA.declare('Union',    ('l','r',  RA))   # union of two sub-programs
RA.declare('Union',   ('l',    RA), ('r',    RA))
RA.declare('Select1',  ('c',      IntSort()), ('p', RA))  # σ_{first=c}(p)
RA = RA.create()

# 2) Set up a small “universe” for testing—say our relation elements are pairs of small ints
#    We'll encode concrete relations as Python sets and then lift them into Z3.
Domain = [(0,0),(0,1),(1,0),(1,1)]
#  We give each pair a distinct Z3 constant so we can refer to them in membership tests:
Pair2 = Datatype('Pair2')
Pair2.declare('mk2', ('f',IntSort()), ('s', IntSort()))
Pair2 = Pair2.create()
# Pre‐make Z3 constants for each domain element
z_pairs = { d : Pair2.mk2(IntVal(d[0]), IntVal(d[1])) for d in Domain }

#  A relation is just a bit-vector indicating which of those 4 pairs are present
Rel   = BitVecSort(len(Domain))

# 3) Define Eval : RA × Rel → Rel by recursion on the AST
Eval = Function('Eval', RA, Rel, Rel)

# 4) Axiomatize Eval for each AST constructor:

xR, yR = Consts('xR yR', Rel)
c      = Int('c')
p      = Const('p', RA)

axioms = []

# 4.a) Table("T1") always returns the *concrete* relation R1
R1 = BitVecVal(0b1010, 4)   # say it contains Domain[1] and Domain[3]
axioms.append(
  ForAll([xR],
    Eval(RA.Table(StringVal("T1")), xR) == R1)
)

# 4.b) Union(p,q)
p1, p2 = Consts('p1 p2', RA)
axioms.append(
  ForAll([p1,p2, xR],
    Eval(RA.Union(p1,p2), xR) ==
      Eval(p1, xR) | Eval(p2, xR))
)

# 4.c) Select1(c, p)
# suppose we have bit-index mapping for each domain element:
index_map = { z_pairs[d]: i for i, d in enumerate(Domain) }

# inside your Sigma1 axiom:
conds = []
for ((c_val, _), z) in z_pairs.items():
    i = index_map[z]
    conds.append(
        And(
            Tuple2.first(x) == c_val,              # x.first == c_val
            Extract(i, i, Eval(p, xR)) == BitVecVal(1,1)
        )
    )
axioms.append(
    ForAll([c, p, xR],
      Eval(RA.Select1(c, p), xR) ==
      If(
        Or(*conds),
        Eval(p, xR),
        BitVecVal(0, len(Domain))
      )
    )
)

# … you’d fill in similar axioms for selection on second field, projection, difference, etc.

# 5) Now *synthesis*:
prog = Const('prog', RA)    # the unknown program we want to find
I0   = R1                   # our single example’s input relation
O0   = BitVecVal(0b1110,4)  # target output for that example

s = Solver()
s.add(axioms)
# Enforce that Eval(prog, I0) == O0
s.add(Eval(prog, I0) == O0)

if s.check() == sat:
    model = s.model()
    print("Found program:", model[prog])
else:
    print("No program of this shape works.")


Z3Exception: Z3 expression expected